### Acquire & Clean Data

In [8]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import pandas as pd
import json
import requests
from dotenv import load_dotenv

%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
cannot find .env file


In [ ]:
def getChannelMessages(access_token, channel_id):
    api_url = "https://discord.com/api/channels/" + channel_id + "/messages"
    api_call_headers = {'Authorization': 'Bot ' + access_token}
    api_call_response = requests.get(api_url, headers=api_call_headers, verify=False)
    
    messages_raw = api_call_response.text
    messages_json = json.loads(messages_raw)
    return messages_json

def jsonToDataframe(messages_json, channel_name):
    timestamps = []
    username = []
    content = []
    for message in messages_json:
        timestamps.append(message['timestamp'])
        username.append(message['author']['username'])
        content.append(message['content'])

    list_of_tuples = list(zip(username, timestamps, content)) 

    df = pd.DataFrame(list_of_tuples,
                      columns = ['username', 'timestamps', 'content']) 
    df['channel'] = channel_name
    return df

In [ ]:
# Convert all text to json
vcj


vc_df = jsonToDataframe(vc, 'vc-discussion')
vc_df.columns
vc_df.head()

In [ ]:
# Cleaning Process
# Remove ""(GIFS), remove the tags between <@!____>
# Fix timestamps 